# Setup

Make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline

In [410]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # for retina screens
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Configure notebook to display all results in cell

In [411]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Revert to the last line of output only
# InteractiveShell.ast_node_interactivity = "last_expr"

# Load Data

In [412]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded_houses_train.csv')
houses_test = pd.read_csv('../Data/encoded_houses_test.csv')

In [413]:
print("houses_train dimensions: {}".format(houses_train.shape))
print("houses_test dimensions: {}".format(houses_test.shape))

houses_train dimensions: (1460, 537)
houses_test dimensions: (1459, 537)


In [414]:
pd.set_option("display.max_columns", 537)
houses_train.head(3)

,MSSubClass30,MSSubClass40,MSSubClass45,MSSubClass50,MSSubClass60,MSSubClass70,MSSubClass75,MSSubClass80,MSSubClass85,MSSubClass90,MSSubClass120,MSSubClass150,MSSubClass160,MSSubClass180,MSSubClass190,MSZoningC (all),MSZoningFV,MSZoningI,MSZoningRH,MSZoningRL,MSZoningRP,MSZoningRM,LotFrontage,LotArea,StreetPave,AlleyPave,AlleyNA,LotShapeIR1,LotShapeIR2,LotShapeIR3,LandContourBnk,LandContourHLS,LandContourLow,UtilitiesNoSewr,UtilitiesNoSeWa,UtilitiesELO,LotConfigCorner,LotConfigCulDSac,LotConfigFR2,LotConfigFR3,LandSlopeMod,LandSlopeSev,NeighborhoodBlueste,NeighborhoodBrDale,NeighborhoodBrkSide,NeighborhoodClearCr,NeighborhoodCollgCr,NeighborhoodCrawfor,NeighborhoodEdwards,NeighborhoodGilbert,NeighborhoodIDOTRR,NeighborhoodMeadowV,NeighborhoodMitchel,NeighborhoodNAmes,NeighborhoodNoRidge,NeighborhoodNPkVill,NeighborhoodNridgHt,NeighborhoodNWAmes,NeighborhoodOldTown,NeighborhoodSWISU,NeighborhoodSawyer,NeighborhoodSawyerW,NeighborhoodSomerst,NeighborhoodStoneBr,NeighborhoodTimber,NeighborhoodVeenker,Condition1Feedr,Condition1Norm,Condition1RRNn,Condition1RRAn,Condition1PosN,Condition1PosA,Condition1RRNe,Condition1RRAe,Condition2Feedr,Condition2Norm,Condition2RRNn,Condition2RRAn,Condition2PosN,Condition2PosA,Condition2RRNe,Condition2RRAe,BldgType2fmCon,BldgTypeDuplex,BldgTypeTwnhsE,BldgTypeTwnhs,HouseStyle1.5Fin,HouseStyle1.5Unf,HouseStyle2Story,HouseStyle2.5Fin,HouseStyle2.5Unf,HouseStyleSFoyer,HouseStyleSLvl,OverallQual9,OverallQual8,OverallQual7,OverallQual6,OverallQual5,OverallQual4,OverallQual3,OverallQual2,OverallQual1,OverallCond9,OverallCond8,OverallCond7,OverallCond6,OverallCond5,OverallCond4,OverallCond3,OverallCond2,OverallCond1,YearBuilt,YearRemodAdd,RoofStyleGable,RoofStyleGambrel,RoofStyleHip,RoofStyleMansard,RoofStyleShed,RoofMatlCompShg,RoofMatlMembran,RoofMatlMetal,RoofMatlRoll,RoofMatlTar&Grv,RoofMatlWdShake,RoofMatlWdShngl,Exterior1stAsphShn,Exterior1stBrkComm,Exterior1stBrkFace,Exterior1stCBlock,Exterior1stCemntBd,Exterior1stHdBoard,Exterior1stImStucc,Exterior1stMetalSd,Exterior1stOther,Exterior1stPlywood,Exterior1stPreCast,Exterior1stStone,Exterior1stStucco,Exterior1stVinylSd,Exterior1stWd Sdng,Exterior1stWdShing,Exterior2ndAsphShn,Exterior2ndBrk Cmn,Exterior2ndBrkFace,Exterior2ndCBlock,Exterior2ndCmentBd,Exterior2ndHdBoard,Exterior2ndImStucc,Exterior2ndMetalSd,Exterior2ndOther,Exterior2ndPlywood,Exterior2ndPreCast,Exterior2ndStone,Exterior2ndStucco,Exterior2ndVinylSd,Exterior2ndWd Sdng,Exterior2ndWd Shng,MasVnrTypeBrkFace,MasVnrTypeCBlock,MasVnrTypeNone,MasVnrTypeStone,MasVnrArea,ExterQualGd,ExterQualTA,ExterQualFa,ExterQualPo,ExterCondGd,ExterCondTA,ExterCondFa,ExterCondPo,FoundationCBlock,FoundationPConc,FoundationSlab,FoundationStone,FoundationWood,BsmtQualGd,BsmtQualTA,BsmtQualFa,BsmtQualPo,BsmtQualNA,BsmtCondGd,BsmtCondTA,BsmtCondFa,BsmtCondPo,BsmtCondNA,BsmtExposureAv,BsmtExposureMn,BsmtExposureNo,BsmtExposureNA,BsmtFinType1ALQ,BsmtFinType1BLQ,BsmtFinType1Rec,BsmtFinType1LwQ,BsmtFinType1Unf,BsmtFinType1NA,BsmtFinSF1,BsmtFinType2ALQ,BsmtFinType2BLQ,BsmtFinType2Rec,BsmtFinType2LwQ,BsmtFinType2Unf,BsmtFinType2NA,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingGasA,HeatingGasW,HeatingGrav,HeatingOthW,HeatingWall,HeatingQCGd,HeatingQCTA,HeatingQCFa,HeatingQCPo,CentralAirY,ElectricalFuseA,ElectricalFuseF,ElectricalFuseP,ElectricalMix,X1stFlrSF,X2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQualGd,KitchenQualTA,KitchenQualFa,KitchenQualPo,TotRmsAbvGrd,FunctionalMin1,FunctionalMin2,FunctionalMod,FunctionalMaj1,FunctionalMaj2,FunctionalSev,FunctionalSal,Fireplaces,FireplaceQuGd,FireplaceQuTA,FireplaceQuFa,FireplaceQuPo,FireplaceQuNA,GarageTypeAttchd,GarageTypeBasment,GarageTypeBuiltIn,GarageTypeCarPort,GarageTypeDetchd,GarageTypeNA,GarageYrBlt,GarageFinishRFn,GarageFinishUnf,GarageFinishNA,GarageCars,GarageArea,GarageQualGd,GarageQualTA,GarageQualFa,GarageQualPo,GarageQualNA,GarageCondGd,GarageCondTA,GarageCondFa,GarageCondPo,GarageCondNA,PavedDriveP,PavedD

In [415]:
houses_train.info()
houses_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 537 entries, MSSubClass30 to SalePrice
dtypes: float64(537)
memory usage: 6.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 537 entries, MSSubClass30 to SalePrice
dtypes: float64(536), int64(1)
memory usage: 6.0 MB


Delete first column

In [416]:
# houses_train.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')
# houses_test.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')

# Run xgboost

### Create private training & test set

In [417]:
from sklearn.model_selection import train_test_split

seed = 10
test_ratio = 0.2

X = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y = houses_train.loc[:, houses_train.columns == "SalePrice"].values

# Take log of SalePrice
y = np.log(y + 1).ravel() # convert to 1D array for model fit (xxx, )


In [418]:
X_pr_train, X_pr_test, y_pr_train, y_pr_test = train_test_split(X, y, test_size=test_ratio, random_state=seed)

In [419]:
print(len(X_pr_train), "train +", len(X_pr_test), "test")

1168 train + 292 test


### Fit Model

In [420]:
from xgboost import XGBRegressor

xgb_clf = XGBRegressor(max_depth=3, 
                        learning_rate=0.1, 
                        n_estimators=1000, # Number of boosted trees to fit
                        silent=False, # print messages while running 
                        objective='reg:linear', 
                        booster='gbtree', # Specify which booster to use: gbtree, gblinear or dart
                        #for dart see http://xgboost.readthedocs.io/en/latest/tutorials/dart.html 
                        n_jobs=-1, # Number of parallel threads used to run xgboost. (replaces nthread)
                        gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                        min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
                        max_delta_step=0, # Maximum delta step we allow each tree’s weight estimation to be
                        subsample=1, # Subsample ratio of the training instance
                        colsample_bytree=1, # Subsample ratio of columns when constructing each tree
                        colsample_bylevel=1, # Subsample ratio of columns for each split, in each level
                        reg_alpha=0, # L1 regularization term on weights
                        reg_lambda=1, # L2 regularization term on weights
                        scale_pos_weight=1, # Balancing of positive and negative weights
                        base_score=0.5, # The initial prediction score of all instances, global bias
                        random_state=743, 
                        missing=None) # Value in the data which needs to be present as a missing value. If None, defaults to np.nan



In [421]:
xgb_clf.fit(X_pr_train, y_pr_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=743,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [422]:
# make predictions for test data

y_pr_pred = xgb_clf.predict(X_pr_test)

Evaluate predictions

In [423]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


0.12132884492961547

In [424]:
y_pr_test[1:5]
y_pr_pred[1:5]

array([ 12.1428719 ,  11.8277435 ,  12.01067193,  12.64109979])

array([ 12.2483902 ,  11.85323715,  11.84269428,  12.66669178], dtype=float32)

Save model to file

In [425]:
from sklearn.externals import joblib # More memory efficient than pickle for large numpy arrays

joblib.dump(xgb_clf, './Models/xgboost_model.pkl') 

['./Models/xgboost_model.pkl']

To load a model:

In [426]:
# xgb_clf_loaded = joblib.load('./Models/xgboost_model.pkl') 

### Model Tuning

In [427]:
from sklearn.model_selection import GridSearchCV

xgb_params = {'max_depth': 3,
              'learning_rate': 0.1, 
              'n_estimators': 100, 
              'objective': 'reg:linear'}

param_grid = {'max_depth': [3, 4, 5], 
              'learning_rate': [0.07], 
              'n_estimators': [1500, 1800, 2000, 2200, 2400], # Number of boosted trees to fit
              'objective': ['reg:linear'], 
              'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
              'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
              'gamma': [0, 0.5],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
              'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
              'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
              'subsample': [1], # Subsample ratio of the training instance
              'colsample_bytree': [0.6], # Subsample ratio of columns when constructing each tree
              'colsample_bylevel': [0.3], # Subsample ratio of columns for each split, in each level
              'reg_alpha': [0], # L1 regularization term on weights
              'reg_lambda': [1], # L2 regularization term on weights
              'scale_pos_weight': [1], # Balancing of positive and negative weights
              'base_score': [0.5], # The initial prediction score of all instances, global bias
              'silent': [True],
              'random_state': [10]}

optimized_xgb_clf = GridSearchCV(XGBRegressor(**xgb_params), # scikit-learn estimator interface 
                                 param_grid = param_grid, # Dictionary with parameters names (string) as keys
                                 scoring="neg_mean_squared_error", # controls what metric they apply to the estimators evaluated
                                 n_jobs=-1, # If True, the data is assumed to be identically distributed across the folds,
                                 iid=True, 
                                 refit=True, # Refit an estimator using the best found parameters (best_estimator_)
                                 cv=3, #integer, to specify the number of folds in a (Stratified)KFold. None -> default 3-fold cross validation
                                 verbose=10, # the higher, the more messages
                                 pre_dispatch="2*n_jobs", # number of jobs that get dispatched during parallel execution
                                 error_score="raise", 
                                 return_train_score=False) #If False, the cv_results_ attribute will not include training scores

In [428]:
# param_grid = {'max_depth': [3], 
#               'learning_rate': [0.05], 
#               'n_estimators': [2000], # Number of boosted trees to fit
#               'objective': ['reg:linear'], 
#               'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
#               'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
#               'gamma': [0],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
#               'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
#               'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
#               'subsample': [1], # Subsample ratio of the training instance
#               'colsample_bytree': [0.6], # Subsample ratio of columns when constructing each tree
#               'colsample_bylevel': [0.3], # Subsample ratio of columns for each split, in each level
#               'reg_alpha': [0], # L1 regularization term on weights
#               'reg_lambda': [1], # L2 regularization term on weights
#               'scale_pos_weight': [1], # Balancing of positive and negative weights
#               'base_score': [0.5], # The initial prediction score of all instances, global bias
#               'silent': [True],
#               'random_state': [10]}

Inspect the grid

In [ ]:
optimized_xgb_clf

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 4, 5], 'learning_rate': [0.07], 'n_estimators': [1500, 1800, 2000, 2200, 2400], 'objective': ['reg:linear'], 'booster': ['gbtree'], 'n_jobs': [-1], 'gamma': [0, 0.5], 'min_child_weight': [1], 'max_delta_step': [0], 'subsample': [1], 'colsample_bytree': [0.6], 'colsample_bylevel': [0.3], 'reg_alpha': [0], 'reg_lambda': [1], 'scale_pos_weight': [1], 'base_score': [0.5], 'silent': [True], 'random_state': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=Fals

Run grid tuning

In [ ]:
optimized_xgb_clf.fit(X_pr_train, y_pr_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1800, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.3, colsample_bytree=0.6, gamma=0, learning_rate=0.07, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=1500, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booste

The best combination of parameters is:

In [ ]:
optimized_xgb_clf.best_params_

In [ ]:
## The best score is
optimized_xgb_clf.best_score_ # that's the training score so not meaningful

In [ ]:
# optimized_xgb_clf.cv_results_

In [ ]:
# make predictions for test data

y_pr_pred = optimized_xgb_clf.predict(X_pr_test)

Evaluate predictions

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


In [ ]:
y_pr_test[1:5]
y_pr_pred[1:5]